## peft quickstart demo

###  AutoModelForSeq2SeqLM introduction

In [19]:
'''
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

    # 指定模型名称或路径
    model_name = "t5-small"

    # 加载预训练模型和分词器
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 输入文本
    input_text = "Translate English to French: How are you?"

    # 编码文本--成模型可接受的输入格式
    inputs = tokenizer(input_text, return_tensors="pt")

    # 生成输出
    outputs = model.generate(**inputs)

    # 解码输出文本
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Input: {input_text}")
    print(f"Output: {output_text}")
'''

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

D:\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yysgz\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Input: Translate English to French: How are you?
Output: Comment êtes-vous?


D:\Anaconda3\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### peft introduction

In [22]:
print([task for task in dir(TaskType) if not task.startswith("__")])

['CAUSAL_LM', 'FEATURE_EXTRACTION', 'QUESTION_ANS', 'SEQ_2_SEQ_LM', 'SEQ_CLS', 'TOKEN_CLS', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [28]:
'''
peft (Parameter-Efficient Fine-Tuning) package introduction:
    Fine-tuning large pretrained models is often prohibitively costly due to their scale. 
    PEFT methods enable efficient adaptation of large pretrained models to various downstream applications by only fine-tuning a 
small number of (extra) model parameters instead of all the model's parameters. 
    This significantly decreases the computational and storage costs. 
    Recent state-of-the-art PEFT techniques achieve performance comparable to fully fine-tuned models.
    PEFT is integrated with Transformers for easy model training and inference, 
peft简化了LLM-finetuning 模型配置和加载功能，特别是使用LoRA等技术。
    - LoraConfig，用于配置LoRA参数。
    - TaskType，用于定义任务类型, e.g. task_type = TaskType.TEXT_GENERATION
    - get_peft_config，用于获取peft配置
    - get_peft_model，用于获取pretrained peft模型。
'''

"\npeft (Parameter-Efficient Fine-Tuning) package introduction:\n    Fine-tuning large pretrained models is often prohibitively costly due to their scale. \n    PEFT methods enable efficient adaptation of large pretrained models to various downstream applications by only fine-tuning a \nsmall number of (extra) model parameters instead of all the model's parameters. \n    This significantly decreases the computational and storage costs. \n    Recent state-of-the-art PEFT techniques achieve performance comparable to fully fine-tuned models.\n    PEFT is integrated with Transformers for easy model training and inference, \npeft简化了LLM-finetuning 模型配置和加载功能，特别是使用LoRA等技术。\n    - LoraConfig，用于配置LoRA参数。\n    - TaskType，用于定义任务类型, e.g. task_type = TaskType.TEXT_GENERATION\n    - get_peft_config，用于获取peft配置\n    - get_peft_model，用于获取pretrained peft模型。\n"

#### get_deft_model

In [37]:
'''
---------------peft 翻译模型----------------------------
# 翻译模型bigscience/mt0-large: English -> French
'''
# prepare a model for training with a PEFT method such as LoRA by wrapping the base model and PEFT configuration with get_peft_model.
# For the bigscience/mt0-large model, you are only training 0.19% of the parameters!
from transformers import AutoModelForSeq2SeqLM  # 用于加载和处理pre-trained seq2seq模型，用于处理nlp任务
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 加载预训练模型和分词器 
model_name = 'bigscience/mt0-large'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 定义lora配置
lora_config = LoraConfig(
    task_type = TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)

# 获取peft model
peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())  # 输出peft mode可训练参数

# 准备输入数据
input_text = "Translate English to French: How are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# 使用 PEFT 模型生成输出
outputs = peft_model.generate(**inputs)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)  # 解码
print(outputs)
print(output_text)

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915
None
tensor([[   0, 6679, 1169,  360,  264, 1465,  259,  291,    1]])
Comment allez-vous?


#### load_peft_model

In [41]:
'''
-----------------peft 因果推理模型--------------------
因果推理模型 ybelkada/opt-350m-lora; gpt2
'''
from peft import AutoPeftModelForCausalLM  # 用于加载和配置因果语言模型Causal LM，并进行高效微调参数
from transformers import AutoTokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoPeftModelForCausalLM.from_pretrained('ybelkada/opt-350m-lora').to(device) 
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-350m')

model.eval()
inputs = tokenizer('Preheat the oven to 350 degrees and place the cookie dough', return_tensors='pt')

outputs = model.generate(input_ids=inputs['input_ids'].to(device), max_new_tokens=50)  # 生成输出
outputs_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # tokenizer解码输出文本
print(outputs)
print(outputs_text)

tensor([[    2, 22763, 25978,     5, 12941,     7, 10088,  4176,     8,   317,
             5, 20931, 14397,    11,     5,  1312,     9,     5, 12941,     4,
         50118, 50118,  1121,    10,   739,  5749,     6,  9637,     5, 15039,
             6, 14814, 10477,     6, 14814, 18833,     6,  6740,     6,     8,
         27053,     4, 50118, 50118,  1121,    10,  2559,  5749,     6,  9637,
             5,  8380,  1423,  1168,  2258,     6,  4696,     6,     8, 21857,
             4, 50118, 50118]])
Preheat the oven to 350 degrees and place the cookie dough in the center of the oven.

In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon.

In a separate bowl, combine the egg yolks, sugar, and vanilla.




## LoRA Network

In [45]:
# peft package implement LoRA: https://github.com/hahuyhoang411/LoRA-Implementation/tree/main

### prompt

In [49]:
from typing import Union

class Prompter(object):
    __slots__=('template')
    
    def __init__(self, template_name: str = '', verbose: bool = False):
        self.template = {
            'description': 'Template used by Alpaca-LoRA.',
            'prompt_input': 'Below is an instruction that describes a task, paired with an input that provides further context. \
             Write a response taht appropriately completes the request.',
            'prompt_no_input': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.',
            'response_split': '### Response:'}
    
    def generate_prompt(self, instruction:str, input: Union[None, str]=None, label: Union[None, str]=None) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template['promt_input'].format(instruction=instruction, input=input)
        else:
            res = self.template['prompt_no_input'].format(instruction=instruction)
        if label:
            res = f"{res}{label}"
        return res
    
    def get_response(self, output: str) -> str:
        return output.split(self.template['response_split'])[1].strip()

### prepare_data

In [50]:
from datasets import load_dataset
import random
from typing import Union  # 类型提示，允许你指定的一个变量可以是多个类型中的任意一种

In [51]:
def create_datasets(data_path, size_valid_set, tokenizer, max_length, seed):
    def tokenize(prompt, add_eos_token=True):
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=max_length,
            padding=False,
            return_tensors=None
        )
        
        if (result['input_ids'][-1] != tokenizer.eos_token_id
           and len(result['input_ids']) < max_length
           and add_eos_token):
            result['input_ids'].append(tokenizer.eos_token_id)
            result['attention_mask'].append(1)
        
        result['labels'] = result['input_ids'].copy()
        return result
    
    def generate_and_tokenize_prompt(data_point):
        full_prompt = prompter.generate_prompt(
            data_point['instruction'],
            data_point['input'],
            data_point['output'],
        )
        tokenized_full_prompt = tokenize(full_prompt)
        
        return tokenized_full_prompt
    
    prompter = Prompter()
    
    print(f'Load dataset...')
    dataset = load_dataset('json', split='train', datafiles=data_path)
    dataset = dataset.train_test_split(test_size=size_valid_set, seed=seed)
    
    train_data = dataset['train'].shuffle().map(generate_and_tokenize_prompt)

### lora model

In [53]:
# lora layer
import math
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F

class LoraLayer:
    def __init__(self, 
                 in_features: int,     # the number of input features
                 out_features: int,):  # the number of output features
        # Initializes dictionaries to store various parameters for each adapter in the layer
        self.r = {}           # the rank of the low-rank matrix
        self.lora_alpha = {}  # the scaling factor
        self.scaling = {}     # the calculated scaling factor (lora_alpha / r)
        
        # Dropout layers for each adapter
        self.lora_dropout = nn.ModuleDict({})
        
        # Weight matrices for the linear layers
        self.lora_A = nn.ModuleDict({})
        self.lora_B = nn.ModuleDict({})
        
        # Weight matrices for the embedding layers
        self.lora_embedding_A = nn.ParameterDict({})
        self.lora_embedding_B = nn.ParameterDict({})
        
        # Boolean flag indicating whether the weights ahve been merged
        self.merged = False
        
        # Boolean flag indicating whether the adapters are disabled
        self.disabled_adapters = False
        
        # Stores the number of input and output features
        self.in_features =  in_features
        self.out_feature = out_features

In [56]:
# lora model
class LoraModel(nn.Module):
    def __init__(self, model, config, adapter_name='default'):
        super().__init__()
        self.model = model
        self.forward = self.model.forward
        self.peft_config = config
        self.add_adapter(adapter_name, self.peft_config[adapter_name])
    
    def add_adapter(self, adapter_name, config=None): pass

### train

### inference

In [57]:
import argparse
import json

import torch
from transformers import GenerationConfig, AutoModelForCausalLM, AutoTokenizer, AutoConfig

In [60]:
def get_response(prompt, tokenizer, model, generation_config, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors='pt')
    output = model.generate(
        input_ids = inputs['input_ids']
        )